# <u><p style="text-align: center;">Extract, Transform, Load (ETL)</p></u>

### Contents of this notebook
* What an ETL procedure is
* ETL case study

### Background

#### ETL

The ***Extract, Transform, Load*** procedure refers to copying data from one or more sources into a destination system which represents the data differently from the source(s). The three actions have the following meanings:
* **Extract**: retrieve data from a source
* **Transform**: convert retrieved data according to rules and lookup tables or create combinations of data from different sources 
* **Load**: store the data in a different location

The ETL procedure is becoming more and more important because we need to handle ever increasing datasets, varying data structures, as well as heterogeneous and multimodal data. 

#### Case study

In this notebook, we will examine the possibility of using a data lake to perform an ETL procedure for transforming and storing sensor data. The selected study was originally embedded into the Big Data project of [BREED4FOOD](https://breed4food.com/). 

![image](https://www.breed4food.com/images/logo.jpg)

The case study accomodated an experiment in which the gait score of 200 turkeys was determined. Traditionally, gait scoring is performed by a trained person. In the study, different types of sensors were used to explore if they can describe the gait score recorded by a trained person. For this notebook, we are going to use only the **Force Plate** sensor.

During the animal study different data types were acquired by each sensor. For the force plate, these were binary files, called Technical Data Management Streaming (TDMS) files. This file format was created to help engineers and scientists to properly store the large amounts of data generated during simulations and tests. 

Here, as a first step we are going to showcase an ETL procedure for a single force plate file. Then, we will scale up our ETL procedure to minimize its execution time when a large number of animals are being investigated.

### Simple ETL for the forceplate data

Before we are able to transform the data it is necessary to load important packages and libraries:

In [ ]:
import os, glob
from nptdms import TdmsFile as td
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql import functions as F

#'swan_spark_conf' is a configuration provided by a plugin for Jupyter. We further extend this configuration with proxy settings.
swan_spark_conf = swan_spark_conf.setAll([('spark.ui.proxyBase', os.environ['JUPYTERHUB_SERVICE_PREFIX'] + 'proxy/4040')])

#instantiate a SparkSession object with our configuration
spark = SparkSession\
            .builder\
            .config(conf=swan_spark_conf)\
            .appName('ETL')\
            .getOrCreate()

#set Spark log level
spark.sparkContext.setLogLevel('ERROR')

#### Extracting the force plate data

Data coming from the forceplate sensor are stored in a proprietary format that Spark cannot read directly. Thus, we need to extract the data and store them in an open fromat, like CSV.

Forceplate data are stored in files with the extention *.tdms*. under a folder named `forceplate_files`. Initially, we generate a list containing the files in the appropriate folder with the extention *.tdms*.

In [ ]:
paths = glob.glob(os.path.join('/home/jovyan/datasets/forceplate_files', '**/*.tdms'))

print(paths)

#### Transforming and loading the data

Then, we iterate through the list and transform the native data to *.csv* format. Finally, we store them in another folder, named `forceplate_csv_files`.

In [ ]:
for path in paths:
    
    print('Source path:', path)
    df = td(path).as_dataframe() #transforming the data into a dataframe
    
    destination_path = '/home/jovyan/datasets/forceplate_csv_files/' + os.path.basename(os.path.dirname(path) + '.csv')
    print('Destination path:', destination_path)
    df.to_csv(destination_path) #loading (storing) the data to a different location as csv files

#### Reading one of the *.csv* files

If you check the *.csv* files, you will notice that the column names are way too long. Thus, we are going to shorten them by manually defining a schema into which Spark will load them. 

In the schema declaration below, we define the column names, and the corresponding data types:

In [ ]:
forceplate_schema = StructType([
    StructField('Time', IntegerType()),
    StructField('Channel1', DoubleType()),
    StructField('Channel2', DoubleType()),
    StructField('Channel3', DoubleType()),
    StructField('Channel4', DoubleType()),
    StructField('Channel5', DoubleType()),
    StructField('Channel6', DoubleType()),
    StructField('Channel7', DoubleType()),
    StructField('Channel8', DoubleType())
])

Now, let's read a *.csv* file, with the schema we set-up earlier:

In [ ]:
channelsDF = spark.read.csv('/home/jovyan/datasets/forceplate_csv_files/18936.csv', header=True, schema=forceplate_schema)

and then inspect the dataframe and its schema:

In [ ]:
channelsDF.schema

We can use the `count` action to see how many data points are in the dataframe:

In [ ]:
channelsDF.count()

and also calculate summary statistics from the dataframe, using the `describe` function:

In [ ]:
channelsDF.describe().show()

### Scaling by loading multiple files at once

The steps for pre-processing data from a single force plate were explained previously. Here, we will try to scale up by processing several files at once. To load one *.csv*  file we used the statement below:

`channelsDF = spark.read.csv('/home/jovyan/datasets/forceplate_csv_files/18936.csv', header=True, schema=forceplate_schema)`

Now, we need to load all the files that are stored in the `forceplate_csv_files` folder. Since we are working with Spark, the only thing we have to change is to replace the *.csv* filename with the folder path of the *.csv* files, and all the CSV files will be read into a single dataframe:

In [ ]:
channelsDFall = spark.read.csv('/home/jovyan/datasets/forceplate_csv_files', header=True, schema=forceplate_schema)

Now, we can check if all files were loaded, for example by printing the total amount of records in the dataframe:

In [ ]:
channelsDFall.count()

which is correct, so the code to load all data is working. Note that this is a massively parallelizable operation that would have worked the same with really big data files stored in a remote location.

For any further processing of the turkeys individually, we would need a way to identify them. The turkey IDs are contained in the filenames as numbers. For example, the file in `/home/jovyan/datasets/forceplate_csv_files/18936.csv` refers to turkey with ID `18936`. However, you may have noticed that except for their name the files do not include an identifier of the turkey they correspond to. Without identifiers our dataframe is impractical for future analysis. This is a common problem in ETL tasks, but Spark has functionality to solve it. For each row in the dataframe, Spark can recall from which file it was loaded, and we can extract the turkey ID information from its path location.

We can do this by using the `withColumn` and `input_file_name` functions, by appending a new column named *input* which contains the *filename* from which each row has been read. The `input_file_name` function returns the full path of the file from which the row has been read, or an empty string if not available. In our case, it contains the turkey identifier (ID). Check the code below and inspect the results of the transformations:

In [ ]:
channelsDFall = channelsDFall.withColumn('input', F.input_file_name())

channelsDFall.select('input').show(3, truncate=False)

Now, we added a column with the path of the file from which the the row has been read. Our next step is to split the ID of each turkey from the path.

To seperate the ID of each turkey we need to split the input string `file:///home/jovyan/datasets/forceplate_csv_files/18936.csv` by /, and keep the part with the turkey identifier. This can be done with the `split` function. From the resulting array we pick the ninth element, which contains the turkey ID:

In [ ]:
channelsDFall = channelsDFall.withColumn('ID', F.split(F.col('input'), '/')[7])
channelsDFall.show(3)

Now, you may have noticed that the file extension (.csv), is still there. To remove it, we can use the `substr` function. With `substr` we can retain the first five digits of the splitted element by extracting a substring. Try the code below and check the output in your ID column:

In [ ]:
channelsDFall = channelsDFall.withColumn('ID', F.split(F.col('input'), '/')[7].substr(0,5))
channelsDFall.show(3)

At this point, we have extracted all the necessary information for further processing. As a final step, we are going to store the transformed data in *.csv* format in a folder named `transformed_forceplate_csv_files`: 

In [ ]:
channelsDFall.write.csv('/home/jovyan/datasets/transformed_forceplate_csv_files/data.csv', header = True)

### Further reading

More information about the case study can be found [here](https://www.sciencedirect.com/science/article/pii/S175173112000155X?via%3Dihub). Also, you can find more material on its [code](https://github.com/ionathan/datalake-locomotion) and [open data](https://zenodo.org/record/3563513) repositories. Other links related to the notebook are below:
* [Data lakes](https://en.wikipedia.org/wiki/Data_lake)
* [ETL procedures](https://en.wikipedia.org/wiki/Extract,_transform,_load)